# relation_df Dataset

Creación de dataset relation_df que buscar relacionar los dataset de google maps y yelp mediante las columnas 'latitude' y 'longitude'

In [14]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [15]:
from pathlib import Path

# Definir la ruta relativa desde el directorio del script
relative_path_business = Path("datasets") / "business.parquet"
relative_path_pizza = Path("datasets") / "pizza_metadata.parquet"

# Cargar el archivo
business = pd.read_parquet(relative_path_business)
pizza_metadata = pd.read_parquet(relative_path_pizza)


In [16]:
business.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,None,93101,34.426679,-119.711197,5.0,7
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,None,63123,38.551126,-90.335695,3.0,15
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,None,85711,32.223236,-110.880452,3.5,22
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,CA,19107,39.955505,-75.155564,4.0,80
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,MO,18054,40.338183,-75.471659,4.5,13


In [17]:
pizza_metadata.head()

,name,address,gmap_id,latitude,longitude,avg_rating,num_of_reviews
0,Sir Pizza Xpress,"Sir Pizza Xpress, 201 E Broad St, Smithville, ...",0x8866cef69c5737a1:0xbdf838584dca0b9c,35.956944,-85.811111,4.1,18
1,Bucci's Greek & Italian Specialties,"Bucci's Greek & Italian Specialties, 8030 S Ho...",0x876c8410e0d7ec6d:0xb06492857fa9ec9d,39.572697,-104.922399,4.7,13
2,Dynamite Pizza & BBQ,"Dynamite Pizza & BBQ, 59069 Gratiot Ave, New H...",0x882518a534093375:0x4333d5f58f7af4c,42.735038,-82.785340,4.1,8
3,Three Star Pizza,"Three Star Pizza, 409 Cabot St #1, Beverly, MA...",0x89e3169821e62d4d:0x14ff0683c1ebca0e,42.559072,-70.881542,3.9,48
4,Hot Stuff Pizza,"Hot Stuff Pizza, 1104 1st St E, Park Rapids, M...",0x52b7b8e4459a8697:0x81e50b7c06d172df,46.921303,-95.037448,3.3,7


Join mediante 'longitude' y 'latitude'

In [18]:
epsilon = 0.0001  # Margen de tolerancia en coordenadas

# Renombrar columnas para evitar conflictos en el merge
business = business.rename(columns={"latitude": "latitude_business", "longitude": "longitude_business"})
pizza_metadata = pizza_metadata.rename(columns={"latitude": "latitude_metadata", "longitude": "longitude_metadata"})

# Merge manteniendo solo los negocios en el dataset business
relation_df = business.merge(
    pizza_metadata,
    how="left"  # Asegura que solo se mantengan los datos de business
)

# Filtrar solo las coincidencias dentro del margen epsilon
relation_df = relation_df.query(
    "abs(latitude_business - latitude_metadata) <= @epsilon and abs(longitude_business - longitude_metadata) <= @epsilon"
)[["name", "business_id", "gmap_id", "latitude_business", "longitude_business"]]

# Renombrar las columnas finales
relation_df = relation_df.rename(columns={"latitude_business": "latitude", "longitude_business": "longitude"})

# Mostrar las primeras filas
relation_df.head()

,name,business_id,gmap_id,latitude,longitude


In [19]:
epsilon = 0.0001  # Margen de tolerancia en coordenadas

relation_df = business.merge(
    pizza_metadata,
    how="inner",
    on=[],
).query(
    "abs(latitude_business - latitude_metadata) <= @epsilon and abs(longitude_business - longitude_metadata) <= @epsilon"
)[["name", "business_id", "gmap_id", "latitude_business", "longitude_business"]]

# Renombrar columnas
relation_df = relation_df.rename(columns={"latitude_business": "latitude", "longitude_business": "longitude"})

# Mostrar resultado
relation_df.head()

IndexError: list index out of range